This code attempts to train convnets from scratch on the final dataset containing 535 images.

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD, Adam

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np
from keras.preprocessing.image import img_to_array, load_img
import os

In [3]:
def load_images_from_folder(direct):
    images = []
    y = []
    for filename in os.listdir(direct):
        image = None
        if filename.endswith('jpeg'):
            img = load_img(path=direct+filename, target_size=(256,256))
        elif filename.endswith('jpg'):
            img = load_img(path=direct+filename, target_size=(256,256))
        if img is not None :
            img = img_to_array(img)
            images.append(img)
            if(filename.startswith('dog')):
                y.append(1)
            else:
                y.append(0)
    return images, y

In [4]:
images, labels = load_images_from_folder('dataset/')

In [5]:
images = np.array(images)
labels = np.array(labels)
print(images.shape)
print(labels.shape)

(534, 256, 256, 3)
(534,)


In [6]:
import sklearn.cross_validation as cv
X_train, X_test, Y_train, Y_test = cv.train_test_split(images, labels, test_size=0.25)

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# I use a single datagenerator to rescale the images in both training and test dataset
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow(X_train, Y_train)
test_generator = datagen.flow(X_test, Y_test)

**Model 1**

In [43]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(256, 256, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3,3), activation='relu'))

model.add(Flatten())  
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 254, 254, 32)      896       
_________________________________________________________________
activation_50 (Activation)   (None, 254, 254, 32)      0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 254, 254, 32)      0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 252, 252, 32)      9248      
_________________________________________________________________
dropout_19 (Dropout)         (None, 252, 252, 32)      0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 250, 250, 64)      18496     
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 125, 125, 64)      0         
__________

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer= SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [44]:
model.fit_generator(train_generator, steps_per_epoch=len(X_train)//16, epochs=50, validation_data=test_generator)

Epoch 1/50
25/25 [==============================] - 24s 946ms/step - loss: 0.7044 - acc: 0.4775 - val_loss: 0.6912 - val_acc: 0.6045
Epoch 2/50
25/25 [==============================] - 22s 873ms/step - loss: 0.6947 - acc: 0.4975 - val_loss: 0.6915 - val_acc: 0.6269
Epoch 3/50
25/25 [==============================] - 22s 886ms/step - loss: 0.6903 - acc: 0.5499 - val_loss: 0.6875 - val_acc: 0.5896
Epoch 4/50
25/25 [==============================] - 22s 861ms/step - loss: 0.6937 - acc: 0.5090 - val_loss: 0.6887 - val_acc: 0.6045
Epoch 5/50
25/25 [==============================] - 22s 873ms/step - loss: 0.6917 - acc: 0.4974 - val_loss: 0.6870 - val_acc: 0.6194
Epoch 6/50
25/25 [==============================] - 22s 890ms/step - loss: 0.6897 - acc: 0.5399 - val_loss: 0.6854 - val_acc: 0.5970
Epoch 7/50
25/25 [==============================] - 22s 875ms/step - loss: 0.6862 - acc: 0.5199 - val_loss: 0.6991 - val_acc: 0.4179
Epoch 8/50
25/25 [==============================] - 21s 855ms/step - 

In [45]:
model.save_weights('new_convnet')

**Validation accuracy = 60-66%**

**Model 2**

In [35]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(256, 256, 3), activation='relu', kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='glorot_normal'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer='glorot_normal'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3,3), activation='relu',kernel_initializer='glorot_normal'))

model.add(Flatten())  
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 125, 125, 32)      9248      
_________________________________________________________________
dropout_18 (Dropout)         (None, 125, 125, 32)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 123, 123, 64)      18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 61, 61, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 59, 59, 128)       73856     
__________

In [36]:
model.compile(loss='binary_crossentropy',
              optimizer= SGD(lr=0.01),
              metrics=['accuracy'])

In [37]:
model.fit_generator(train_generator, steps_per_epoch=len(X_train)//16, epochs=50, validation_data=test_generator)

Epoch 1/50
25/25 [==============================] - 6s 257ms/step - loss: 0.6948 - acc: 0.5287 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 2/50
25/25 [==============================] - 5s 218ms/step - loss: 0.6881 - acc: 0.5374 - val_loss: 0.6943 - val_acc: 0.5000
Epoch 3/50
25/25 [==============================] - 5s 215ms/step - loss: 0.6826 - acc: 0.5399 - val_loss: 0.6949 - val_acc: 0.4925
Epoch 4/50
25/25 [==============================] - 5s 218ms/step - loss: 0.6835 - acc: 0.5598 - val_loss: 0.6961 - val_acc: 0.5000
Epoch 5/50
25/25 [==============================] - 5s 217ms/step - loss: 0.6759 - acc: 0.5774 - val_loss: 0.6948 - val_acc: 0.5000
Epoch 6/50
25/25 [==============================] - 5s 218ms/step - loss: 0.6724 - acc: 0.6025 - val_loss: 0.6998 - val_acc: 0.4925
Epoch 7/50
25/25 [==============================] - 6s 221ms/step - loss: 0.6653 - acc: 0.6013 - val_loss: 0.7008 - val_acc: 0.4776
Epoch 8/50
25/25 [==============================] - 5s 218ms/step - loss: 0.

In [38]:
model.save_weights('new_convnet2.h5')

**Validation accuracy = 65%**

I tried various combinations of learning rates, initializers, optimizers, dropout rates and architectures.